In [1]:
import laser_lib
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

In [2]:
queue = laser_lib.DacQueue()

Found  1 Helios DACs


In [3]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


## Concentric, Rotating, Discrete Beam Circles


In [4]:
# Start w/ one circle
T = 1200
colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]


arr_pos1 = np.zeros((T, 2))
arr_pos2 = np.zeros((T, 2))

arr_col = np.zeros((T, 3))


# Color the 
num_beams = 20
dt = T//num_beams
arr_col[0::dt, :] = colors[0]
arr_col[20::dt, :] = colors[1]
arr_col[40::dt, :] = colors[2]

# Lower the dac rate
#queue.dac_rate = 20000

for step in range(1000):
    phase = 2*np.pi*step/1000.0
    theta1 = np.linspace(phase+0, phase+2*np.pi, T)
    theta2 = np.linspace(0-phase, 2*np.pi-phase, T)
    
    arr_col1 = np.zeros((T, 3))
    arr_col2 = np.zeros((T, 3))
    
    # Pos a circle
    arr_pos1[:, 0] = np.cos(theta1)
    arr_pos1[:, 1] = np.sin(theta1)
    arr_col1[arr_pos1[:, 1] > 0] = arr_col[arr_pos1[:, 1] > 0]
    queue.submit(arr_pos1, arr_col1)
    
    arr_pos2[:, 0] = np.cos(theta2)
    arr_pos2[:, 1] = np.sin(theta2)
    
    arr_col2[arr_pos2[:, 1] > 0] = arr_col[arr_pos2[:, 1] > 0]
    queue.submit(0.75*arr_pos2, arr_col2)
    queue.submit(0.50*arr_pos1, arr_col1)


KeyboardInterrupt: 

## Concentric circles w/ dwell and blanking

In [5]:
# SPIRAL PATTERN (DEEP INSIDE MY BRAIN)
num_beams = 18
# Make sure to keep dwell/blank small and close together for good performance
dwell = 60
blank = 55
num_colors = 3
T = num_beams*dwell
# Stratified theta
theta = np.linspace(0, 2*np.pi, num_beams, endpoint=False)
# Taking stratified theta and repeat it dwell times so we stop at those points
theta = np.repeat(theta, dwell)

# Set dac rate
queue.dac_rate = 30000

arr_pos = np.zeros((T, 2))

colors = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
# Repeat rgb -> rrrgggbbb
colors = np.repeat(colors, dwell, axis=0)
# Tile = Repeat the entire color pattern end over end
arr_col = np.tile(colors, (num_beams//num_colors, 1))

# Blank out the laser while mirrors moving. Laser displays for dwell-blank timestamps
for b in range(blank):
    arr_col[b::dwell, :] = 0
    
phase = 0
delta = -0.2*np.pi
lim1 = 0 + 10*delta
lim2 = np.pi + delta + 0.1
lim3 = 1.9*np.pi

for i in range(1000):
    theta_new = (theta + phase)%(2*np.pi)
    arr_pos[:, 0] = np.cos(theta_new)
    arr_pos[:, 1] = np.sin(theta_new)
    # Mask based off of theta, not in the x-y space. theta = (0, 2pi)
    #mask = np.logical_or(np.logical_and(theta_new > lim1, theta_new < lim2), theta_new > lim3)
    #queue.submit(arr_pos, arr_col*mask[:, np.newaxis])
    queue.submit(arr_pos, arr_col)
    
    #theta_new = (theta - phase)%(2*np.pi)
    #arr_pos[:, 0] = 0.75*np.cos(theta_new)
    #arr_pos[:, 1] = 0.75*np.sin(theta_new)
    #mask = np.logical_or(np.logical_and(theta_new > lim1, theta_new < lim2), theta_new > lim3)
    #queue.submit(arr_pos, arr_col*mask[:, np.newaxis])
    
    #theta_new = (theta + phase)%(2*np.pi)
    #arr_pos[:, 0] = 0.5*np.cos(theta_new)
    #arr_pos[:, 1] = 0.5*np.sin(theta_new)
    #mask = np.logical_or(np.logical_and(theta_new > lim1, theta_new < lim2), theta_new > lim3)
    #queue.submit(arr_pos, arr_col*mask[:, np.newaxis])
    
    phase += 0.01*np.pi


KeyboardInterrupt: 

In [ ]:
print(np.repeat(np.array([1, 2, 3]), 3))
print(np.tile(np.array([1, 2, 3]), 3))
print(np.arange(10)[1::3])

In [ ]:
def beamify(arr_pos_smooth, arr_col, num_beams, blank=50):
    '''
    Takes a smooth trajectory and makes it into individual beams
    Also performs blanking on the color array 
    '''
    # Find how often to sample the original position array
    T = arr_pos_smooth.shape[0]
    dt = T//num_beams
    
    # Sample the position array and repeat it 
    beam_points = arr_pos_smooth[::dt, :]
    beam_points = np.repeat(beam_points, num_beams, axis=0)
    
    # Make a new pos arr and set all of them to the last beam's pos
    arr_pos_beamed = np.zeros_like(arr_pos_smooth)
    arr_pos_beamed[:, :] = beam_points[-1]
    arr_pos_beamed[0:beam_points.shape[0], :] = beam_points
    
    # Adds blanking to the color array
    for b in range(blank):
        arr_col[b::dt] = 0
        
    for i in range(100):
        print("FIX THIS FUNCTION ")
        return 0
        
    return arr_pos_beamed, arr_col
    
    

In [14]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)
arr_col = np.ones((T, 3))

arr_pos, arr_col = beamify(arr_pos, arr_col, num_beams=30, blank=10)

for i in range(100):
    queue.submit(arr_pos, arr_col)

NameError: name 'beamify' is not defined

In [34]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

In [ ]:
# Frames -> Patterns -> Sequences

In [ ]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
while(True):
    for i in range(num_steps-30):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
        i -= 20
        arr_col_out = arr_col_out*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)

    for i in range(num_steps-30):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
        i -= 20
        arr_col_out = arr_col_out*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)


In [39]:
for i in range(10):
    print(i)
    i += 20
    print(i)

0
20
1
21
2
22
3
23
4
24
5
25
6
26
7
27
8
28
9
29


In [ ]:
thet = 2
x = 0.5

In [ ]:
theta + pi/ 2 > x & theta + pi/ 2 < x

In [ ]:
x = np.array([True, True, False, False])
y = np.array([True, False, True, False])
print(np.logical_or(x, y))
print(np.logical_and(x, y))

In [ ]:
print(queue.dac_rate)

In [25]:
# Time of the entire pattern
Ts = 6.0
# Points per frame
T = 1000
# Time per frame
Tf = T/queue.dac_rate
# Time for each segment
T_seg = 1.0

num_frames = int(Ts/Tf)
print(num_frames)
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_col = np.ones((T, 3))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)
for frame_idx in range(num_frames):
    queue.submit(arr_pos, arr_col)
    

    

180


In [6]:
Tf

0.03333333333333333